# Debias

Perform a basic *monthly debiasing* on GDPS and compare it to the raw model

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import dask
import dask.array as da
import dask.dataframe as dd
import dask.distributed
import dask_jobqueue
import numpy as np
import os
import pandas as pd
import pathlib
import plotly.express as px

In [ ]:
DATA_DIR = pathlib.Path(os.getenv("DATA_DIR"))
INPUT_DATASET = DATA_DIR / pathlib.Path('2021-12-20-gdps-metar/')

In [ ]:
input_path = pathlib.Path(INPUT_DATASET)

In [ ]:
sample_path = next(iter(input_path.glob('*.parquet')))

In [ ]:
sample = pd.read_parquet(sample_path)

In [ ]:
columns = set(sample.columns)
columns -= set(['gdps_hpbl'])

GDPS HPBL is unavailable from 2019010100 to 2019031100. I think it's because the field was thinned out before we asked for it to be dearchived.
The partially available column causes problems in this notebook, so I remove the column.
Eventually we should make sure a gdps_hpbl with nans is available in all the dataframes.

In [ ]:
cluster = dask_jobqueue.SLURMCluster(
    env_extra=['source ~/.bash_profile','conda activate smc01'],
    name='smc01-dask',
)

In [ ]:
cluster.scale(jobs=4)

In [ ]:
client = dask.distributed.Client(cluster)

In [ ]:
client

In [ ]:
dataset_files = sorted(list(input_path.glob('*.parquet')))

In [ ]:
df = dd.read_parquet(dataset_files, columns=columns)

In [ ]:
df.head()

In [ ]:
df = df.reset_index()

In [ ]:
df['date'].max().compute()

In [ ]:
df['step_hour'] = df['step'] / 3600
df['error_2t'] = df['obs_2t'] - df['gdps_2t']
df['squared_error_2t'] = (df['gdps_2t'] - df['obs_2t']) ** 2
df['rmse_2t'] = da.sqrt(df['squared_error_2t'])
df['mabs_2t'] = np.abs(df['error_2t'])
df['forecast_month'] = df['date'].dt.month

In [ ]:
df = df.set_index('date')

In [ ]:
df['step_td'] = dd.to_timedelta(df['step'], unit='S')

In [ ]:
df['step_td']

In [ ]:
train = df.loc['2019-01-01':'2021-01-01']
val = df.loc['2021-01-01':'2022-01-01']

In [ ]:
monthly_bias = train.groupby(['station', 'forecast_month', 'step_hour']).agg({'error_2t': 'mean'}).compute()

In [ ]:
monthly_bias = monthly_bias.rename(columns={'error_2t': 'bias_2t'})

In [ ]:
monthly_bias = monthly_bias.reset_index()

In [ ]:
train = train.merge(monthly_bias, on=['station', 'forecast_month', 'step_hour'])

In [ ]:
train['gdps_corrected_2t'] = train['gdps_2t'] + train['bias_2t']

In [ ]:
train['gdps_corrected_2t_error'] = train['obs_2t'] - train['gdps_corrected_2t']

In [ ]:
train['gdps_corrected_2t_se'] = train['gdps_corrected_2t_error'] ** 2

In [ ]:
errors = train.groupby('step_hour').agg({'gdps_corrected_2t_se': 'mean', 'squared_error_2t': 'mean'}).compute()

In [ ]:
errors = errors.reset_index().melt(id_vars=['step_hour'])

In [ ]:
px.line(data_frame=errors, x='step_hour', y='value', color='variable')

In [ ]:
monthly_bias

In [ ]:
val = val.merge(monthly_bias, on=['station', 'forecast_month', 'step_hour'])

In [ ]:
val['gdps_corrected_2t'] = val['gdps_2t'] + val['bias_2t']
val['gdps_corrected_2t_error'] = val['obs_2t'] - val['gdps_corrected_2t']
val['gdps_corrected_2t_se'] = val['gdps_corrected_2t_error'] ** 2

In [ ]:
errors = val.groupby('step_hour').agg({'gdps_corrected_2t_se': 'mean', 'squared_error_2t': 'mean'}).compute()


In [ ]:
errors

In [ ]:
melt_errors = errors.reset_index().melt(id_vars=['step_hour'])

In [ ]:
px.line(data_frame=melt_errors, x='step_hour', y='value', color='variable')

In [ ]:
errors.columns

In [ ]:
errors['skill_score'] = 1.0 - (errors['gdps_corrected_2t_se'] / errors['squared_error_2t'])

In [ ]:
errors

In [ ]:
px.line(data_frame=errors, x=errors.index, y='skill_score')

The performance of a monthly debiasing model decreases pretty linearly with time.
I suspect all of our models will have the same phenomenon.
Next model to try is probably an EMOS model, but this will be a little more involved.
